In [1]:
!pip install diaparser

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.1/342.1 kB 11.1 MB/s eta 0:00:00


In [2]:
texts = []

with open('/kaggle/input/mahabharat-index/cleanedTextFileFinalDONE.txt', 'r',encoding='utf-8') as file:
    for line in file:
        texts.append(line[:-1])
        


In [3]:
def extract_rel_window(text_list, keywords_list):
    sents = []
    for sent in text_list: 
        for keyword in keywords_list:
            if keyword in sent:
                index = sent.find(keyword)
                min_idx = index
                max_idx = index
                
                while min_idx>=0 and sent[min_idx]!='.':
                    min_idx = min_idx-1
                while max_idx<len(sent) and sent[max_idx]!='.':
                    max_idx = max_idx+1
                
                # max_idx = len(sent)-1
                # if index+window_size<len(sent)-1:
                #     max_idx = index+window_size
                min_idx = min_idx + 1
                sents.append(sent[min_idx:max_idx])
    return sents

In [4]:
def remove_before_last_alphabet(sentence):
    # Find the index of the last alphabet
    last_alpha_index = None
    for i in range(len(sentence) - 1, -1, -1):
        if sentence[i].isalpha():
            last_alpha_index = i
            break
    
    # Remove everything before the last alphabet
    if last_alpha_index is not None:
        sentence = sentence[0:last_alpha_index+1]
    
    # Add a full stop at the end
    sentence = sentence.strip() + "."
    
    return sentence

def assign_token_ids(sent):
    # Initialize an empty list to store token information
    tokens = []

    # Iterate over each token in the sentence
    for i, (word, head, rel) in enumerate(zip(sent.words, sent.arcs, sent.rels), start=1):
        # Extract relevant information from each token
        token_info = {
            "id": i,
            "word": word,
            "head": head,
            "rel": rel,
        }
        tokens.append(token_info)

    return tokens

import nltk
nltk.download('wordnet')

from nltk.corpus import wordnet

def is_english_word(word):
    # Check if the word is in WordNet's list of English words
    return wordnet.synsets(word) != []

def transliterate_to_latin(word):
    iast_mapping = {
        'á': 'aa',
        'ñ': 'n',
        'ā': 'aa',
        'ă': 'aa',
        'ī': 'i',
        'ĭ': 'i',
        'Ś': 'sh',
        'ś': 'sh',
        'Ū': 'u',
        'ū': 'u',
        'ḍ': 'd',
        'Ḣ': '',
        'ḥ': '',
        'ṃ': 'm',
        'ṅ': 'n',
        'ṇ': 'n',
        'ṛ': 'ri',
        'ṝ': 'ri',
        'ṣ': 'sh',
        'ṭ': 't'
    }

    latin_word = ''
    for char in word:
        if char in iast_mapping:
            latin_word += iast_mapping[char]
        else:
            latin_word += char
    return latin_word





[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
sents_wife = extract_rel_window(texts,["wife of"])
sents_wife = [sent.lower() for sent in sents_wife]
sents_wife

['aśmaka1, a rājarṣi, son of vaśiṣṭha and madayantī, the wife of kalmāṣapāda, ',
 'aśmakī, a yādavī, the wife of king prācinvat and mother of saṃyāti, ',
 'adṛśyantī, wife of śaktri  and daughter in law of vasiṣṭha, and mother of parāśara   ',
 'ahalyā, the wife of the ṛṣi gautama ',
 'aikṣvākī, a princess, wife of suhotra, and mother of ajamīḍha  ',
 ' tvāṣṭrī, the wife of savitṛ, in the form of a mare, gave birth to the aśvins in the skies',
 'amṛtā, a māgadha princess, wife of anaśvan ',
 'anantā, a mādhava princess, wife of pūrus son janamejaya ',
 'āṅgī, an ancient queen, wife of ariha and mother of mahābhauma ',
 ' niśā was likewise the wife of bhānu, and gave birth to one daughter, to agni and soma, and to five fires ',
 'arundhatī, the wife of vasiṣṭha  b, a',
 ' b, the wife of atri abaned her husband and sought the protection of mahā deva, who granted her that she should obtain a son without atri, ',
 '  gaṅgā agreed to become the wife of śāntanu on the condition that she woul

In [6]:
from diaparser.parsers import Parser
parser = Parser.load('en_ewt.electra-base')

Downloading: "https://github.com/Unipisa/diaparser/releases/download/v1.0/en_ewt.electra-base" to /root/.cache/diaparser/en_ewt.electra-base
100%|██████████| 452M/452M [00:01<00:00, 261MB/s] 


config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [13]:
text = "vinatā, daughter of daksa and wife of kaśyapa, mother of the vainateyas. "
dataset = parser.predict(text, text='en')
dataset.sentences[0]

# sent_id = 1
# text = vinatā, daughter of daksa and wife of kaśyapa, mother of the vainateyas.
1	vinatā	_	_	_	_	0	root	_	_
2	,	_	_	_	_	1	punct	_	_
3	daughter	_	_	_	_	1	appos	_	_
4	of	_	_	_	_	5	case	_	_
5	daksa	_	_	_	_	3	nmod	_	_
6	and	_	_	_	_	7	cc	_	_
7	wife	_	_	_	_	1	conj	_	_
8	of	_	_	_	_	9	case	_	_
9	kaśyapa	_	_	_	_	7	nmod	_	_
10	,	_	_	_	_	9	punct	_	_
11	mother	_	_	_	_	9	appos	_	_
12	of	_	_	_	_	14	case	_	_
13	the	_	_	_	_	14	det	_	_
14	vainateyas	_	_	_	_	11	nmod	_	_
15	.	_	_	_	_	1	punct	_	_

In [11]:
text = "sujātā, daughter of uddālaka, wife of kahoda, and mother of astāvakra. "
dataset = parser.predict(text, text='en')
dataset.sentences[0]

# sent_id = 1
# text = sujātā, daughter of uddālaka, wife of kahoda, and mother of astāvakra.
1	sujātā	_	_	_	_	0	root	_	_
2	,	_	_	_	_	1	punct	_	_
3	daughter	_	_	_	_	1	appos	_	_
4	of	_	_	_	_	5	case	_	_
5	uddālaka	_	_	_	_	3	nmod	_	_
6	,	_	_	_	_	7	punct	_	_
7	wife	_	_	_	_	5	conj	_	_
8	of	_	_	_	_	9	case	_	_
9	kahoda	_	_	_	_	7	nmod	_	_
10	,	_	_	_	_	12	punct	_	_
11	and	_	_	_	_	12	cc	_	_
12	mother	_	_	_	_	1	conj	_	_
13	of	_	_	_	_	14	case	_	_
14	astāvakra	_	_	_	_	12	nmod	_	_
15	.	_	_	_	_	1	punct	_	_

In [17]:
text = "ghṛtācī, an apsaras, wife of pramati "
dataset = parser.predict(text, text='en')
dataset.sentences[0]

# sent_id = 1
# text = ghṛtācī, an apsaras, wife of pramati
1	ghṛtācī	_	_	_	_	0	root	_	_
2	,	_	_	_	_	1	punct	_	_
3	an	_	_	_	_	4	det	_	_
4	apsaras	_	_	_	_	1	appos	_	_
5	,	_	_	_	_	1	punct	_	_
6	wife	_	_	_	_	1	appos	_	_
7	of	_	_	_	_	8	case	_	_
8	pramati	_	_	_	_	6	nmod	_	_

In [18]:
text = "kumārī, wife of the elder bhīmasena "
dataset = parser.predict(text, text='en')
dataset.sentences[0]

# sent_id = 1
# text = kumārī, wife of the elder bhīmasena
1	kumārī	_	_	_	_	0	root	_	_
2	,	_	_	_	_	1	punct	_	_
3	wife	_	_	_	_	1	appos	_	_
4	of	_	_	_	_	7	case	_	_
5	the	_	_	_	_	7	det	_	_
6	elder	_	_	_	_	7	amod	_	_
7	bhīmasena	_	_	_	_	3	nmod	_	_

In [24]:
#TEMP
def extract_rel(sents,rel):
    ans_dict = []
    my_dict = {}
    found_relations=[]
    
    
    # sentences_10 = [ "He explained to him why kine have descended to earth In days of yore, when in the Deva yuga the Dānavas became lords of the three worlds, Aditi underwent the severest austerities and begot Viṣṇu seeing Aditi perform penances, Surabhī, the daughter of Dakṣa, also underwent austerities on Kailāsa, in yoga, standing upon one leg for, years Ṛ and M."]
    
    for sent in sents:
        if len(sent)<5:
            continue
        sent = remove_before_last_alphabet(sent)
        sent = transliterate_to_latin(sent)
        a = assign_token_ids((parser.predict(sent, text='en')).sentences[0])
        # finding head 
        head=""
        id=0
        for i in a:
    #         print(i)
            if i['word']==rel:
                id  = i['id']
                head_pointer = i['head']
                for j in a:
                    if j['id']==head_pointer:
                        head = j['word']
    
        if head=="":
            continue
            
        # finding tail 
        tails = []
        tail = ""
        for i in range(id+1 , len(a)):
            if a[i]['rel'] != 'nmod' or a[i]['word'] == 'king' or a[i]['word'] == 'queen':
                tail = tail + a[i]['word'] + " "
            else:
                tails.append(tail+a[i]['word'])
                j=i+1
                #find multiple tails if possible:
                while(j<len(a) and (a[j]['word'] in ['and',','] or a[j]['rel'] == 'conj')):
                    if a[j]['word']=='and' or a[j]['word']==',':
                        del a[j]
                    elif a[j]['rel'] == 'conj':
                        tails.append(a[j]['word'])
                        j=j+1
                    
                break

        tails = [tail for tail in tails if (len(tail.split()) < 6)]
        
        # print(head , 'father ', tail , 'text : ', sent,'\n')
        # found_relations.append(head + ' father ' + tail )

        for tail in tails:
            my_dict = {}
            my_dict['head']=head
            my_dict['relation']=rel+' of'
            my_dict['tail']=tail
            my_dict['text']=sent
            ans_dict.append(my_dict)
            
    
    return ans_dict    
    
    
# printing answers

# for i in found_relations:
#     print(i)


    
    
# print(found_relations)
# print(ans_dict)

In [25]:
len(sents_wife)

201

In [26]:
rels = extract_rel(sents_wife,'wife')

In [27]:
rels

[{'head': 'vashishtha',
  'relation': 'wife of',
  'tail': 'kalmaashapaada',
  'text': 'ashmaka1, a raajarshi, son of vashishtha and madayanti, the wife of kalmaashapaada.'},
 {'head': 'ashmaki',
  'relation': 'wife of',
  'tail': 'king praacinvat',
  'text': 'ashmaki, a yaadavi, the wife of king praacinvat and mother of samyaati.'},
 {'head': 'ashmaki',
  'relation': 'wife of',
  'tail': 'mother',
  'text': 'ashmaki, a yaadavi, the wife of king praacinvat and mother of samyaati.'},
 {'head': 'adrishyanti',
  'relation': 'wife of',
  'tail': 'shaktri',
  'text': 'adrishyanti, wife of shaktri  and daughter in law of vasishtha, and mother of paraashara.'},
 {'head': 'adrishyanti',
  'relation': 'wife of',
  'tail': 'daughter',
  'text': 'adrishyanti, wife of shaktri  and daughter in law of vasishtha, and mother of paraashara.'},
 {'head': 'ahalyaa',
  'relation': 'wife of',
  'tail': 'the rishi',
  'text': 'ahalyaa, the wife of the rishi gautama.'},
 {'head': 'princess',
  'relation': 'w

In [28]:
import pickle

with open('WIFE.pkl','wb') as f:
    pickle.dump(rels,f)
    

# Husband relations


In [29]:
sents_hus = extract_rel_window(texts,["husband of"])
sents_hus = [sent.lower() for sent in sents_hus]
sents_hus

['adhiratha, a sūta in the neighbourhood of campā and friend of dhṛtarāṣṭra husband of rādhā and foster father of karṇa, whom he called vasuṣeṇa and sent to hāstinapura to be instructed by droṇa  at the show of arms he enters the arena to congratulate karṇa as king of aṅga     ',
 'śiva1, the great god, husband of umā             brahmaṇaḥ sutaḥ, had promulgated the pāśupata system brahma rudra saṃv',
 'droṇa, the preceptor of the dhārtarāṣṭras and the pāṇḍavas, dhṛṣṭadyumna, etc son of bharadvāja, husband of kṛpī, and father of aśvatthāman     ',
 'duṣyanta1, an ancient king, the husband of śakuntalā and the father of bharata ',
 'dyumatsena, king of the śālvas, husband of śaibyā, and father of satyavat    ',
 ' additionsthe husband of śacīsahasrākṣaḥvajrahastaḥsahasrā kṣaṃ, c',
 'janamejaya1, son of parikṣit and mādravatī, and the husband of vapuṣṭamā at his snake sacrifice vaiśampāyana recited the mhbhr',
 'jaratkāru1, a ṛṣi, husband of jaratkāru and father of āstīka',
 'husband of 

In [30]:
text = "vasudeva, son of śūra, husband of devakī, etc father of kṛṣṇa, etc. "
dataset = parser.predict(text, text='en')
dataset.sentences[0]

# sent_id = 1
# text = vasudeva, son of śūra, husband of devakī, etc father of kṛṣṇa, etc.
1	vasudeva	_	_	_	_	0	root	_	_
2	,	_	_	_	_	1	punct	_	_
3	son	_	_	_	_	1	appos	_	_
4	of	_	_	_	_	5	case	_	_
5	śūra	_	_	_	_	3	nmod	_	_
6	,	_	_	_	_	7	punct	_	_
7	husband	_	_	_	_	1	list	_	_
8	of	_	_	_	_	9	case	_	_
9	devakī	_	_	_	_	7	nmod	_	_
10	,	_	_	_	_	12	punct	_	_
11	etc	_	_	_	_	12	advmod	_	_
12	father	_	_	_	_	1	list	_	_
13	of	_	_	_	_	14	case	_	_
14	kṛṣṇa	_	_	_	_	12	nmod	_	_
15	,	_	_	_	_	16	punct	_	_
16	etc.	_	_	_	_	1	list	_	_

In [31]:
text = "virāta, king of the matsyas, husband of sudeṣṇā, father of  śaṅkha, uttara, and uttarā."
dataset = parser.predict(text, text='en')
dataset.sentences[0]

# sent_id = 1
# text = virāta, king of the matsyas, husband of sudeṣṇā, father of  śaṅkha, uttara, and uttarā.
1	virāta	_	_	_	_	0	root	_	_
2	,	_	_	_	_	1	punct	_	_
3	king	_	_	_	_	1	appos	_	_
4	of	_	_	_	_	6	case	_	_
5	the	_	_	_	_	6	det	_	_
6	matsyas	_	_	_	_	3	nmod	_	_
7	,	_	_	_	_	8	punct	_	_
8	husband	_	_	_	_	1	list	_	_
9	of	_	_	_	_	10	case	_	_
10	sudeṣṇā	_	_	_	_	8	nmod	_	_
11	,	_	_	_	_	12	punct	_	_
12	father	_	_	_	_	1	list	_	_
13	of	_	_	_	_	14	case	_	_
14	śaṅkha	_	_	_	_	12	nmod	_	_
15	,	_	_	_	_	16	punct	_	_
16	uttara	_	_	_	_	14	conj	_	_
17	,	_	_	_	_	19	punct	_	_
18	and	_	_	_	_	19	cc	_	_
19	uttarā	_	_	_	_	14	conj	_	_
20	.	_	_	_	_	1	punct	_	_

In [33]:
rels_hus = extract_rel(sents_hus,"husband")

In [34]:
rels_hus

[{'head': 'friend',
  'relation': 'husband of',
  'tail': 'raadhaa',
  'text': 'adhiratha, a suta in the neighbourhood of campaa and friend of dhritaraashtra husband of raadhaa and foster father of karna, whom he called vasushena and sent to haastinapura to be instructed by drona  at the show of arms he enters the arena to congratulate karna as king of anga.'},
 {'head': 'god',
  'relation': 'husband of',
  'tail': 'umaa brahmana suta',
  'text': 'shiva1, the great god, husband of umaa             brahmana suta, had promulgated the paashupata system brahma rudra samv.'},
 {'head': 'bharadvaaja',
  'relation': 'husband of',
  'tail': 'kripi',
  'text': 'drona, the preceptor of the dhaartaraashtras and the paandavas, dhrishtadyumna, etc son of bharadvaaja, husband of kripi, and father of ashvatthaaman.'},
 {'head': 'bharadvaaja',
  'relation': 'husband of',
  'tail': 'father',
  'text': 'drona, the preceptor of the dhaartaraashtras and the paandavas, dhrishtadyumna, etc son of bharadvaaj

In [36]:
import pickle

with open('HUSBAND.pkl','wb') as f:
    pickle.dump(rels_hus,f)
    